# A Script to batch convert videos stills into meshs

    Using Colmap to do the conversion from image to mesh and post-proccessing in houdini to clean up, 
    color and stadaize the output objects, saving them as .ply


In [ ]:
import subprocess
import shlex
import os
import time
import glob



Path = str(os.getcwd())




 ### Running Colmap through a  .sh script
 
     Colmap is not accessible through python so here we run a .sh script which we pass the directory of 
     the stills. Colmap outpust a sparse and dense construct of the scene both used later int the procces,
     as well as a delaunay triangulated mesh.

In [ ]:
startTime = time.time()

#execute colmap photogrametry through .sh file
shScript = "./get_colmap_reconstruction.sh" 
totFiles = glob.glob(Path+'/Rocks/*/')
num=0


print("Colabmap sparse+dense conversion begin:")
for rockdir in totFiles:
    print("Rock Dir:")
    print(rockdir)
    rockdir = str(rockdir)

    images = rockdir
    wrkspc = rockdir 

    num+=1

    timer = time.time()

    fh = open("NUL","w")
    subprocess.call([shScript, images, wrkspc], stdout = fh, stderr = fh)
    fh.close()

    print("Colmap convert:")
    print(str(num)+"of"+str(len(totFiles)))
    print("Time in Mins:")
    print((time.time()-timer)/60)


## Cleaning the delaunay mesh  with houdini

    Using Houdini to remove the baseplate from the rock meshes and smoothing the noise of the scanned
    mesh. Using attribute transfer to add the colour from the fused.ply created in the dense reconstruct to 
    colourize the the delaunay meshed model. This code simply loads the models into houdini at the start of 
    a network of nodes and saves the outcome from the botom of the tree, quickly and efficiently utilizing 
    the procedural powers of Houdini for batch proccess.

In [ ]:

import HouProccess
num=0
#execute houdini skinner for each file 
print("Houdini refine begin:")
for rockdir in totFiles:
    print("Rock Dir:")
    print(rockdir)	

    timer = time.time()
    denseDir = rockdir + "/dense/0"
    name = os.path.basename(os.path.normpath(rockdir))
    HouProccess.convert( denseDir+"/fused.ply", denseDir+"/meshed.ply", rockdir+"/"+name+"_Mesh+Col.ply")
    num+=1

    print("Houdini mesh Processed:")
    print(num,"of",len(totFiles))
    print("Time in Mins:")
    print((time.time()-timer)/60)

print("COMPLETE!!!!")
print("processed in", startTime-time.time())
print("PHweeeee")
